# Assignment: Weatherbot Development

(Original : https://github.com/mdd01/Rasa/tree/27340d5bd52a1be28462418f956e6828b2e6f174/Rasa-course-files-master/OneDrive_Tech2000/weatherbot ) 

## Task:

- By completing this assignment you will be able to build weatherbot using Rasa AI stack
- Follow the overview and steps below to create your own weatherbot

## Overview:

- This is how the architecture of the weatherbot looks
- Users asks for weather condition in London
- Text is processed in Rasa NLU (Natural Language Understanding), where the models identify the intent and entities
- Then Rasa Core responds with the action corresponding to the itent of the user input
- If the user asks for weather condition of a location, Rasa Core uses an API call to retrieve the weather conditions in that location and outputs the message as shown in the text box

## Steps to create `weatherbot`
![centered](images/bot-development-process.png)

## Installation and setup

Go to `weatherbot` directory

```
cd weatherbot
```

Install all the requirements for `weatherbot`:

```
pip install -r requirements.txt
```

## Building or enhancing data for NLU

Intent classification and entity extraction are machine learning tasks and all predictive models need training data. To date, we have seen data stored in `JSON` format<br/>

There are two methods of creating the training data:
1. Writing NLU data manually (what we have done so far, and is _time consuming and tedious_)
2. rasa-nlu-trainer (online tool to generate data which can be used to train the chatbot)


### NLU data for weatherbot

- The data file can be found: `weatherbot/data/data.json`
- As with other NLU training data we have viewed, this file stores the `intent` and `entity` of the user input

![max-height-300](images/data.png)

### Visualize weatherbot stories

- To visualize the weatherbot stories as graph we follow the same steps for installation we used earlier in the day
- To review, here is what was run:

```
sudo apt-get install graphviz
dot -V
alias graphviz='dot'
sudo apt-get install python-dev graphviz libgraphviz-dev pkg-config
pip install pygraphviz
```

### Produce graph
- Run the `rasa_core` visualization function to create graph
- Make sure that your current directory includes `weather_domain.yml` and a data folder which would have `stories.md`

```
python -m rasa_core.visualize -d weather_domain.yml -s data/stories.md -o graph.png
```

![max-height-500](images/graph.png)

### Activity: Things to look out for:
- How many stories are in this graph?
- What did those stories look like conversationally?
- What entities were relevant?

### Activity - weatherbot repo exploration

- Inspect the structure of the folder with

```
ls -al
```

### Overall file structure in repo:
 - Which files are relevant to NLU?
 - Which files are relevant to Core?
 - Which files are just nice to haves?

### NLU Questions
- How many intents are there?
- How many entities can it recognize?
- What pipeline elements is it using?

### Core questions
- How many stories are in the training data?
- What actions are there?

### Configuration file

* Configuration file provides important parameters for training the model. We will be using the sklearn based pipeline - spacy_sklearn pipeline. 
* This has already been created for you, named `config_spacy.json`, and it is located in the weatherbot folder. `.weather/config_spacy`
![centered](images/config_spacy.png)

![centered](images/assess-nlu-data.png)

### Training data structure and format
Data file location `./weatherbot/data/data.json`
![centered](images/data_file.png)

Training Data for chatbots is stored in `JSON` format. You can find the file in data folder by the name `data.json`. `JSON` stands for JavaScript Object Notation, it is a human readable text notation using attribute-value pair. All data is stored in attribute "rasa_nlu_data". Every user input or custom examples will be stored as value of attribute text. Then we will have the intent and entity of the text stored in subsequent attributes. Intent and entity are identified using the nlu_model.py script which will be expalined to you shortly. 

* **Important Note:** In case you want to edit the YML files, never use tab for indenting! 

### Enhance NLU

* Since we want a robust chatbot, we need **more data**
* You can use GUI tool provided by Rasa:

_**Rasa NLU trainer**_

### Accessing Rasa NLU trainer
* Rasa NLU trainer is a useful tool to create training data for chatbots
* You can access rasa-nlu-trainer by executing following command on your terminal 

```
 rasa-nlu-trainer
```
* The online trainer should open in your default browser. If your default browser is not Google Chrome, copy the URL and open the trainer in Chrome as it works best in that browser.

### Interactive NLU training data generation tool
![max-height-500](images/nlu-train-data-tool.png)

Within it you can easily label entities along with categorizing them by intent

Once the added content is there, you can easily download the updated file. 

### Save changes

* You can go ahead and save the added data to your existing `.weather/data/data.json` file 
  by clicking "Save" in the top right corner
* Location of the data file is also shown in the top left corner

## Prepare to create NLU model from our data

### `nlu_model.py` First part - import
We start by importing the necessary libraries from the rasa_nlu module
 
```python
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer
from rasa_nlu.model import Metadata, Interpreter
```

### `nlu_model.py` Second part - `def train_nlu`

This function will train the model and store it in the model directory

```python
def train_nlu(data, configs, model_dir):
    training_data = load_data(data)
    trainer = Trainer(config.load(configs))
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir, fixed_model_name = 'weathernlu')
```

Then we define a `train_nlu` function with data file (`data.json`), configuration file and model directory where all model files will be saved. Function loads the data file using the `load_data` method of `rasa_nlu.training_data`, initialize trainer which stores the `Trainer` class from `rasa_nlu` which trains the model using the configuration file. Then call the trainer's method train on the `training_data` file. At last we can define the `model_directory` by calling the trainer's method called persist and give `model_dir` and model name as parameters.

### `nlu_model.py` Third part - `def run_nlu`

Our second function is `run_nlu`, which parses the user input using Interpreter's method `parse`

```python
def run_nlu():
    interpreter = Interpreter.load('./models/nlu/default/weathernlu')
    print(interpreter.parse("I am planning my holiday to LA. I wonder what is the weather 
    out there"))

```

### `nlu_model.py` Put it all together and call it

Now we create `main` function that calls the `train_nlu` function with appropriate arguments

```python
if __name__ == '__main__':
    train_nlu('./data/data.json', 'config_spacy.json', './weatherbot/models/nlu')
    run_nlu()
```

## Run `nlu_model.py` to train NLU model for weather bot

Run the `.weather/nlu_model.py` from your terminal to get intent classification and entity extraction output. Make sure you are in the `weatherbot` folder. 
```
python nlu_model.py
```

## Examining the Core data for enhancement

### First we will look at the files relevant to the Core

Now we will work on  the python file `.weather/actions.py` 

### Rasa Core components of Weather Bot

* Dialogue management model will predict actions/response chatbot should make using the machine learning models, based on the context and state of the conversation
* This allows the chatbot to keep a natural conversation flow and ensure better user experience

###  Domain file 
`.weather/weather_domain.yml`

* As explained previously, domain is the universe where chatbot lives and operates
* Define the 5 key parts of a domain:


1. Slots
2. Intent
3. Entities
4. Templates 
5. Actions

Our dialogue management model would use this data to decide what action should be executed next.

![centered](images/domain_shot.png)

* Slots are the placeholders where you can store things you want to track during the conversation
* Since we have only text data, type of slot would be `text`
* We have three intents defined here `greet`, `goodbye` and `inform`
* And only `entity` defined is `location`

![centered](images/domain2.png)

* Templates are text responses from chatbot once the specific actions are predicted 
  
  For example: 
  When the action predicted is a greeting (**utter_greet**), then bot would respond
     **'Hello! How can I help?'**
     
* You need to create a custom action in Python, which will be used to make an API call to create response message when we ask for weather conditions of a specific location



* Import the libraries from `__future__` package  to ensure the new language is compatible with the interpreter

```python
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

from rasa_core.actions.action import Action
from rasa_core.events import SlotSet
```

* You should have a class `ActionWeather` defined with parameter `Action` which is being called in `weather_domain`
* Whenever `action_weather` is predicted, the chatbot will run everything in this class
* You need to get your API keys to authenticate the API calls. You can sign in https://www.apixu.com/ and get free API key
* Once you have the key, you become an authenticated client to retrieve weather updates from the website ActionWeather class in `.weatherbot/weather_domain.yml` 
* This file makes an API call to get the weather conditions of the specified location

```python
class ActionWeather(Action):
    
    def name(self):
        return 'action_weather'
    
    def run(self, dispatcher, tracker, domain):
        from apixu.client import ApixuClient
        api_key = '...' #your apixu key from https://www.apixu.com/
        client = ApixuClient(api_key) 

        loc = tracker.get_slot('location') #loc will be used keep track of locations 
        mentioned by the users throughout the converstaion
        current = client.getCurrentWeather(q=loc) #we use the client and the
        method getCurrentWeather with location as paramater.
```            

- The response of this API call will be a Python `dictionary`
- You can retrieve the country name, city name, weather condition, temperature in
  celsius, humidity, wind_mph
- You can use the `dispatcher` from `rasa_core` with method `utter_message` having the parameters
  as `message` and finally 
- We can return the current slot value of the `location`

```python
        country = current['location']['country']
        city = current['location']['name']
        condition = current['current']['condition']['text'] 
        temperature_c = current['current']['temp_c']
        humidity = current['current']['humidity']
        wind_mph = current['current']['wind_mph']

        response = """It is currently {} in {} at the moment. The temperature 
        is {} degrees, the humidity is {}% and the wind speed is
        {} mph.""".format(condition, city, temperature_c, humidity, wind_mph)
         
        dispatcher.utter_message(response)
        return [SlotSet('location',loc)]
```    

Example response to an API call

![max-height-500](images/apixu.png)

 *  Save the `actions.py` file after you have updated it with your API key
 *  Now the class `ActionWeather` call from the `weather_domain.yml` is executable, so it can make an API call to get the weather conditions of the specified location

### Stories 

* As explained earlier, training data sample for the dialogue system is called a `story`
* A new `story` starts with `##` followed by a name, which can be used for debugging
* Snapshot below shows how a conversation is being stored in the form of a story

### Note:

**Entities** in NLU fill in the **Slots** in Core Dialogue Actions

![max-height-300](images/new-vocab-slots.png)

### Key point - stories reference previous actions and intents

- The bot was able to retrieve the weather for Italy because it had the `location` slot
- But before that it had to ask for the location because the user hadn't given the location
- The user only asked for the weather

![max-height-300](images/core-story-sequence.png)



# Dialogue model overview

### Inputs to Rasa Core dialogue model

The dialogue and actions of the bot are determined by two main inputs

![max-height-500](images/core-inputs.png)

###  Output of Rasa core dialogue model

The core then uses the `KerasPolicy` of a neural network, frequently a Long Short Term Memory (LSTM) NN, to predict the next appropriate action

![max-height-500](images/core-output.png)

### KerasPolicy chooses the action with the highest probability

- The LSTM handles the classification problem "out of all the potential actions, which one is most likely correct?"
- The LSTM assigns probabilities to each potential action and the `KerasPolicy` chooses the action with the highest probability

![max-height-500](images/LSTM-core.png)

### Output becomes next input

The dialogue model then uses the current output to inform the next action's input

![max-height-500](images/output-to-input.png)

### Review core vocabulary

- **State** = previous action of the bot, used as an input to assign future actions
- **Policy** = the model or function you are using to predict and assign actions
- **LSTM** = a particular type of recurrent neural network; Keras policy uses an LSTM to train on the stories to generate predictions for actions

### The role every component plays in the overall bot architecture

![max-height-500](images/rasa_architecture.png)

### Creating custom data - stories

* You will create custom stories using the interactive learning tool provided by Rasa Core. 
* Your conversation with your bot can be exported to `.data/stories.md` 
* You will then be using the two training Python files, 
 - `.weatherbot/train_init.py`
 - `.weatherbot/train_online.py`

### Examine scripts to train Core

`train_init.py`

* You will use predefined models of `rasa_core` on our training data
* Our training data will be `stories.md`

```python
import logging
from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy

if __name__ == '__main__':
    logging.basicConfig(level='INFO')
    training_data_file = './data/stories.md' 
    model_path = './models/dialogue' 
```

### Policy Options: standard Rasa AI Keras Policy

#### This script uses two different policies:
- `KerasPolicy` which works with neural networks
- `Memoization` which works with a dictionary of keys and entries of conditionals and actions

#### Sometimes another policy will be included:
- `FallbackPolicy`


### Script for Fallback Policy 

If a fallback policy is included, it would appear in the script like so:


```python
...
from rasa_core.policies.fallback import FallbackPolicy
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.agent import Agent

fallback = FallbackPolicy(fallback_action_name="action_default_fallback",
                          core_threshold=0.3,
                          nlu_threshold=0.3)

agent = Agent("domain.yml",
               policies=[KerasPolicy(), fallback])
...
```


### Fallback Policy scenario

![max-height-500](images/fallback1.png)

### Fallback policy evaluates and implements action

- Policy evaluates prediced probabilities at a `core_threshold=0.3`
- If not probability is greater than `0.3`, the `action_default_fallback` is used

![max-height-500](images/fallback2.png)

- Note:`action_fallback` is a default action in Rasa Core, which will send the `utter_default` template message to the user
- Make sure to specify this template in your domain file
- It will also revert back to the state of the conversation before the user message that caused the fallback, so that it will not influence the prediction of future actions. You can take a look at the source of the action below:
    - `class rasa_core.actions.action.ActionDefaultFallback` 
- It Executes the fallback action and goes back to the previous state of the dialogue

### Specifications within script

* We are using the `Agent` class from `rasa_core` with `MemoizationPolicy` and `KerasPolicy` 
* The Keras Policy being used is an LSTM model, you can tune the parameters, add or remove layers etc.

```python
    agent = Agent('weather_domain.yml', 
                  policies = [MemoizationPolicy(max_history = 2),
                  KerasPolicy()]) 

    agent.train(training_data_file, 
                epochs = 500,
                batch_size = 10, 
                validation_split = 0.2)

    agent.persist(model_path)
```    

### Train core model
Now run this script in your terminal.
```
python train_init.py
```
This would train the model with 500 epochs. An epoch describes the number of times the algorithm sees the entire dataset. So, each time the algorithm has seen all samples in the dataset, an epoch has completed.

```
...
Using TensorFlow backend.
WARNING:rasa_core.agent:Passing a file name to `agent.train(...)` is deprecated. Rather load the data with `data = agent.load_data(file_name)` and pass it to `agent.train(data)`.
Processed Story Blocks: 100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 3745.58it/s, # trackers=1]
...
```

### The Keras model has the following structure: 
```
...
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
masking_1 (Masking)          (None, 5, 11)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                5632      
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
_________________________________________________________________
activation_1 (Activation)    (None, 6)                 0         
=================================================================
Total params: 5,830
Trainable params: 5,830
Non-trainable params: 0
_________________________________________________________________
...
```

### Keras LSTM specification:

Network architecture of LSTM has four layers
1. Masking
2. LSTM  
3. Dense
4. Activation

- The output shape of each layer is defined in the `rasa_core.keras_policy.py`.
- Parameters for a layer are calculated as follows:

```python
num_params = [(num_units + input_dim + 1) * num_units] * number_of_layers
```


```
...
Epoch 500/500
31/31 [==============================] - 0s - loss: 0.0052 - acc: 1.0000 - val_loss: 0.1900 - val_acc: 0.8750
INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Persisted model to '/Users/praneeshkhanna/weatherbot/Weatherbot_Tutorial/weatherbot/models/dialogue'
...
```
The model is saved in the `models/dialogue directory`

### The script we run to initialize the interactive training is `train_online.py` 

Let us inspect the first section of `train_online.py`

```python
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
```

### Next, we import methods to run an actual chatbot conversation

We need additional methods from `rasa_core` as the online session is an actual conversation with the chatbot
 - `ConsoleInputChannel`, `RegexInterpreter`, and `RasaNLUInterpreter` function together to read input text from the console and classify intent with entities
 - `Agent`, `KerasPolicy`, and `MemoizationPolicy` should look familiar from developing our Core model

```python
import logging

from rasa_core.agent import Agent
from rasa_core.channels.console import ConsoleInputChannel
from rasa_core.interpreter import RegexInterpreter
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.interpreter import RasaNLUInterpreter
```

### Much of this shoud look similar to `train_init.py`
We are using `agent` the same way as we did in `train_init.py` with additional parameters like `input_channel` and `interpreter`

```python
logger = logging.getLogger(__name__)


def run_weather_online(input_channel, interpreter,
                       domain_file="weather_domain.yml",
                       training_data_file='data/stories.md'):
    
    agent = Agent(domain_file,
                  policies=[MemoizationPolicy(max_history=2), KerasPolicy()],
                  interpreter=interpreter)
                  
    agent.train_online(training_data_file,
                       input_channel=input_channel, 
                       batch_size=50, 
                       epochs=200, 
                       max_training_samples=300)
    return agent

if __name__ == '__main__':
    logging.basicConfig(level="INFO")
    nlu_interpreter = RasaNLUInterpreter('./models/nlu/default/weathernlu')
    run_weather_online(ConsoleInputChannel(), nlu_interpreter)
```

### To initiate the interactive training, use the script `train_online.py`

- You can interactively add new stories and train the dialogue model
- Run this script on your terminal

```
python train_online.py
``` 

### Once initiated, the script outputs the parameters of the dialogue model:

Architecture of the `train_online.py` model.
```
...
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
masking_1 (Masking)          (None, 5, 14)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                6016      
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 231       
_________________________________________________________________
activation_1 (Activation)    (None, 7)                 0         
=================================================================
Total params: 6,247
Trainable params: 6,247
Non-trainable params: 0
_________________________________________________________________
...

```

### And `train_online.py` outputs the baseline accuracy of the core model

The model should run for 200 epochs, outputting the loss value and accuracy
```
Epoch 200/200
39/39 [==============================] - 0s - loss: 0.5101 - acc: 0.8205
INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
```

### Begin to interactively train the bot

- When the bot is loaded and you can start the conversation

```
Bot loaded. Type a message and press enter: 
Hello
```

### This is the start of the training conversation

```bash
------
Chat history:

	bot did:	None  

	bot did:	action_listen

	user said:	Hello

		 whose intent is:	greet

we currently have slots: location: None

------
The bot wants to [action_listen] due to the intent. Is this correct?

	1.	Yes
	2.	No, intent is right but the action is wrong
	3.	The intent is wrong
	0.	Export current conversations as stories and quit

```

Note that the bot is tracking previous actions on both parts and provides options for directing the bot

### At each point, you can confirm if the bot acted as expected
* Weather Bot correctly predicts the intent as greet, so the corresponding action is listen, so we respond 1.
* Now we can ask about weather conditions of a city we like

```
1
Next user input:
How's the weather in Chicago ?
```

### Proceed to next part of the training conversation
```
------
Chat history:

	bot did:	None

	bot did:	action_listen

	user did:	greet

	bot did:	action_listen

	user said:	How's the weather in Chicago ?

		 whose intent is:	inform

	with location:	chicago

we currently have slots: location: chicago

------
The bot wants to [action_weather] due to the intent. Is this correct?

	1.	Yes
	2.	No, intent is right but the action is wrong
	3.	The intent is wrong
	0.	Export current conversations as stories and quit
```

### Once you confirm it does the correct action, the bot will respond accordingly
- Respond 1 as `action_weather` is the correct response 
- Weatherbot will respond with weather conditions in selected location

```
1
It is currently Partly cloudy in Chicago at the moment. The temperature is 27.2 degrees, the humidity is 49% and the wind speed is 6.9 mph.
------
```

### Bot still wants to check if it should do anything next

```
------
Chat history:

	bot did:	action_listen

	user did:	greet

	bot did:	action_listen

	user did:	inform

	with location:	chicago

	bot did:	action_weather

we currently have slots: location: chicago

------
The bot wants to [action_listen]. Is this correct?

	1.	Yes.
	2.	No, the action is wrong.
	0.	Export current conversations as stories and quit
    
```

### Exit the conversation and export the updated stories

- Choose the third option 0 because we want to export the stories generated to our Markdown file `./weatherbot/stories.md` 

```
0
File to export to (if file exists, this will append the stories) [stories.md]:
```

- Give the location where `stories.md` should be saved

```
.data/stories.md
```

- You should see this message on your terminal

```
INFO:rasa_core.policies.online_trainer:Stories got exported to './weatherbot/.data/stories.md'.
```


### Check - the generated story is now in `stories.md`

- `.weatherbot/data/stories.md` should now contain the conversation you just had with the bot

![centered](images/new_convo.png)

### Activity - add two more stories to the Weather Bot

### Retrain core model with new story

* Now you need to retrain the model with the new stories added
* Run the `.weatherbot/train_init.py` in your terminal to retrain the model

```
python train_init.py
```


### Retrained model output

- Observing from the last epoch, validation accuracy on `./weather/train_init.py` has increased from 87.5% to 100% on adding new data

```
Epoch 500/500
43/43 [==============================] - 0s - loss: 0.0085 - acc: 1.0000 - val_loss: 0.0061 - val_acc: 1.0000
INFO:rasa_core.policies.keras_policy:Done fitting keras policy model

```

### Revisit the original visualization graph

- Visualize and see the difference in the graphs from before

```
python -m rasa_core.visualize -d weather_domain.yml -s data/stories.md -o graph.png
```

### Review graph after story customization
You can notice the new branch added to the graph.
![max-height-500](images/graph_new1.png)

* To visualize the customized stories, we would need to run the same command we used earlier
* Make sure you can see the exported story in `.data/stories.md` file before executing the following command


## Last script to run Weather Bot

The last script we need to execute and understand is `dialogue_management_model.py`

### Examine `dialogue_management_model.py`

- Import all the libraries and initialize the logger for debugging

```python
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import logging

from rasa_core.agent import Agent
from rasa_core.channels.console import ConsoleInputChannel
from rasa_core.interpreter import RegexInterpreter
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.interpreter import RasaNLUInterpreter

logger = logging.getLogger(__name__)
```

### Examine `dialogue_management_model.py`
Examine functions in script

```python
def train_dialogue(domain_file = 'weather_domain.yml',
                   model_path =  './weatherbot/models/dialogue',
                   training_data_file = './weatherbot/data/stories.md'):
    
    agent = Agent(domain_file, policies = [MemoizationPolicy(), KerasPolicy()])
    
    agent.train(training_data_file,
                epochs = 300,
                batch_size = 50,
                validation_split = 0.2)

    agent.persist(model_path)
    return agent
```

* Note: You can tweak model parameters and see if you can improve validation accuracy

### Examine `dialogue_management_model.py`

Examine functions in script

```python
def run_weather_bot(serve_forever=True):
    interpreter =RasaNLUInterpreter('./weatherbot/models/nlu/default/weathernlu')
    agent = Agent.load('.weatherbot/models/dialogue', interpreter = interpreter) 
    if serve_forever:
        agent.handle_channel(ConsoleInputChannel())
        return agent   
if __name__ == '__main__':
    train_dialogue()
    run_weather_bot()
```    

* The second function of the script is used to check if the server is constantly listening to
  the messages. 
* You can load the `rasa_nlu` model using `RasaNLUInterpreter` and the agent using `load` method of `Agent` class
* Now you can start listening the incoming messages using `handle_channel` method
* Finally you can call the `train_dialogue` and `run_weather_bot` function

### Run script to run bot

- Run the script on your terminal

```
python dialogue_management_model.py
```

### Check dialogue model output

```
Epoch 300/300
43/43 [==============================] - 0s - loss: 0.2026 - acc: 1.0000 - val_loss: 0.2934 - val_acc: 0.9091

/Users/[user-name]/anaconda3/lib/python3.6/site-packages/rasa_nlu/extractors/entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/nlu/default/weathernlu/entity_synonyms.json'
  "".format(entity_synonyms_file))

```

### Interact with bot

```
Bot loaded. Type a message and press enter: 
```

Hello

```
Hello! How can I help?
```

How's the weather in Seattle?

```
It is currently Sunny in Seattle at the moment. The temperature is 26.1 degrees, the humidity is 44% and the wind speed is 0.0 mph.
```